In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe

import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import java.util.UUID.randomUUID

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val ACCOUNT_MATCHERS_CSV_FILE = "../../data/provision/account-matchers.csv"
val FUND_MATCHERS_CSV_FILE = "../../data/provision/fund-matchers.csv"

// TODO(Johann) create a user
val userId = randomUUID()

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

In [2]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(userId).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(userId, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=f1ce9839-ed41-42bc-8476-ca2c14be290e, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=ff98b717-bfaf-43fc-b351-f52b73d8c457, name=ING RON, unit=Currency(value=RON))
AccountTO(id=d4847669-5d39-474b-b7ce-666a44b7a8ff, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=d7b4a65e-f57c-4835-82e2-60209d5c6e14, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=2b2f218a-10b5-4d47-9112-a0b516b303c4, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=e0e1d6b2-71fd-4d3b-aa3d-3d551f619d67, name=Other, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    // TODO(Johann) fix npe on get existing funds without accounts
    val existingFunds = fundSdk.listFunds(userId).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(userId, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=913c2c12-6ea3-44fe-881c-bb42f5b595df, name=Expenses)
FundTO(id=e5fbcc72-7a82-4fcc-8515-601b33d3a41d, name=Work Income)
FundTO(id=3884a765-9747-4a32-94b5-7c570bc3ba55, name=Savings)

In [5]:
import ro.jf.funds.importer.api.model.AccountMatcherTO

val accountMatchers = runBlocking {
    DataFrame.readCSV(ACCOUNT_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row -> AccountMatcherTO(row.getValue<String>("import_account_name"), AccountName(row.getValue<String>("account_name"))) }
}
accountMatchers.map { it.toString() }.joinToString(separator = "\n")

AccountMatcherTO(importAccountName=ING old, accountName=ING RON)
AccountMatcherTO(importAccountName=ING Economy old, accountName=ING Economy)
AccountMatcherTO(importAccountName=Cash RON, accountName=Cash RON)
AccountMatcherTO(importAccountName=Food Coupons, accountName=Food Coupons)
AccountMatcherTO(importAccountName=Revolut - Manual, accountName=Revolut RON)
AccountMatcherTO(importAccountName=Other, accountName=Other)

In [6]:
import ro.jf.funds.importer.api.model.*

val fundMatchers = runBlocking {
    DataFrame.readCSV(FUND_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_account" -> FundMatcherTO.ByAccount(
                    importAccountName = row.getValue<String>("import_account_name"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )
                
                "by_label" -> FundMatcherTO.ByLabel(
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label" -> FundMatcherTO.ByAccountLabel(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label_with_transfer" -> FundMatcherTO.ByAccountLabelWithTransfer(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    initialFundName = FundName(row.getValue<String>("initial_fund_name")),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                else -> error("fund matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByAccountLabelWithTransfer(importAccountName=ING Economy old, importLabel=Invest Profit Acc, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING old, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabel(importLabel=Gifts, fundName=Expenses)
ByAccount(importAccountName=ING old, fundName=Expenses)
ByAccount(importAccountName=ING Economy old, fundName=Expenses)
ByAccount(importAccountName=Cash RON, fundName=Expenses)
ByAccount(importAccountName=Other, fundName=Exp

In [8]:
import kotlinx.coroutines.runBlocking
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*

// TODO(Johann) load all files from the directory
val csvFiles = listOf(
    File("../../data/wallet/2019/wallet_export_2019_01.csv"),
    File("../../data/wallet/2019/wallet_export_2019_02.csv"),
    File("../../data/wallet/2019/wallet_export_2019_03.csv"),
)
val importResponses = csvFiles.forEach { file ->
    val importConfiguration = ImportConfigurationTO(
        fileType = ImportFileTypeTO.WALLET_CSV,
        accountMatchers = accountMatchers,
        fundMatchers = fundMatchers
    )
    runBlocking { importSdk.import(userId, importConfiguration, csvFiles) }
}


ro.jf.funds.commons.error.ApiException: ApiException: 500 - ErrorTO(title=Internal error, detail=ApiException: 500 - ErrorTO(title=Internal error, detail=ApiException: 200 - ErrorTO(title=Generic error, detail={"items":[{"id":"dc852390-2ea5-47cd-991a-50e70ebde34f","dateTime":"2019-01-31T02:00:49","records":[{"id":"45f7f96c-ff6c-4d72-b26a-e97b7bac958b","accountId":"ff98b717-bfaf-43fc-b351-f52b73d8c457","amount":-13.8,"unit":{"type":"currency","value":"RON"},"metadata":{"fundId":"913c2c12-6ea3-44fe-881c-bb42f5b595df"}}],"metadata":{}},{"id":"904ee339-cbe8-4f7b-bc1a-bfd109eb769d","dateTime":"2019-01-31T02:00:42","records":[{"id":"2e6c0015-a3d9-423d-bb02-0ff2001f2a2f","accountId":"ff98b717-bfaf-43fc-b351-f52b73d8c457","amount":-5.0,"unit":{"type":"currency","value":"RON"},"metadata":{"fundId":"913c2c12-6ea3-44fe-881c-bb42f5b595df"}}],"metadata":{}},{"id":"363e495d-8d10-4b5c-aa9f-a69ae26b4f62","dateTime":"2019-01-30T02:00:36","records":[{"id":"c539d458-c225-400c-a652-6d98e42b6263","accountI